# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [56]:
# import libraries
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import nltk
#nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import pickle
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import nltk
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
print(os.getcwd())

/Users/davideffiong/Documents/Disaster_Response_Pipepline


In [58]:

# load data from database
engine = create_engine('sqlite:////Users/davideffiong/Documents/Disaster_Response_Pipepline/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM disaster_table", engine)

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
X = df['message']

Y = df.iloc[:,4:]


### 2. Write a tokenization function to process your text data

In [60]:
print(X[0])

Weather update - a cold front from Cuba that could pass over Haiti


In [61]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
lemmatizer

<WordNetLemmatizer>

In [62]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [63]:
print(tokenize(X[0]))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [55]:
df.shape

(26216, 40)

In [66]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7febc04c8e60>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [85]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

In [72]:
Y_preds = pipeline.predict(X_test)

In [86]:
results_dict = {}

for pred, label, col in zip(Y_preds.transpose(), Y_test.values.transpose(), Y_test.columns):
    print(col)
    print(classification_report(label, pred))
    results_dict[col] = classification_report(label, pred, output_dict=True)

related
              precision    recall  f1-score   support

           0       0.26      0.15      0.19      1882
           1       0.76      0.86      0.80      5937
           2       0.01      0.02      0.02        46

    accuracy                           0.68      7865
   macro avg       0.34      0.34      0.34      7865
weighted avg       0.63      0.68      0.65      7865

request
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      6506
           1       0.16      0.10      0.12      1359

    accuracy                           0.76      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.71      0.76      0.73      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7835
           1       0.00      0.00      0.00        30

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7865

    accuracy                           1.00      7865
   macro avg       1.00      1.00      1.00      7865
weighted avg       1.00      1.00      1.00      7865

water
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      7339
           1       0.06      0.02      0.03       526

    accuracy                           0.91      7865
   macro avg       0.50      0.50      0.49      7865
weighted avg       0.87      0.91      0.89      7865

food
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      7015
           1       0.10      0.07      0.09       850

    accuracy                           0.83      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.81      0.83      0.82      7865

shelter
              precision    recall  f1-scor

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      7382
           1       0.00      0.00      0.00       483

    accuracy                           0.94      7865
   macro avg       0.47      0.50      0.48      7865
weighted avg       0.88      0.94      0.91      7865

transport
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      7504
           1       0.02      0.00      0.00       361

    accuracy                           0.95      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.91      0.95      0.93      7865

buildings
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      7442
           1       0.02      0.00      0.00       423

    accuracy                           0.94      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.90      0.

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.